In [1]:
!pwd

/home/jobquiroz/full_stack_deep_learning/lab02/notebooks


In [2]:
# Fixing path
import os

os.getcwd()   # Verify where it is right now...

'/home/jobquiroz/full_stack_deep_learning/lab02/notebooks'

In [3]:
# Solution, go to lab directory:
os.chdir('/home/jobquiroz/full_stack_deep_learning/lab02/')

In [4]:
os.listdir()

['text_recognizer',
 'notebooks',
 'lightning_logs',
 'training',
 '.ipynb_checkpoints',
 'Untitled.ipynb']

### CNN on synthethic handwritting data

In [10]:
import random
import IPython.display as display

randsize = 10 ** (random.random() * 2 + 1)

Url = "https://fsdl-public-assets.s3.us-west-2.amazonaws.com/emnist/U.png"

# run multiple times to display the same image at different sizes
#  the content of the image remains unambiguous
display.Image(url=Url, width=randsize, height=randsize)

In [12]:
import torch

generic_linear_transform = torch.randn(8, 1)
print("generic:", generic_linear_transform, sep="\n")

local_linear_transform = torch.tensor([
    [0, 0, 0] + [random.random(), random.random(), random.random()] + [0, 0]]).T
print("local:", local_linear_transform, sep="\n")

generic:
tensor([[ 0.6151],
        [-1.2588],
        [ 1.4434],
        [ 0.1989],
        [-1.3881],
        [ 1.1812],
        [ 1.2038],
        [-0.8398]])
local:
tensor([[0.0000],
        [0.0000],
        [0.0000],
        [0.0089],
        [0.0369],
        [0.0386],
        [0.0000],
        [0.0000]])


In [14]:
generic_linear_transform = torch.arange(8)[:, None]
print("generic:", generic_linear_transform, sep="\n")

equivariant_linear_transform = torch.stack([torch.roll(generic_linear_transform[:, 0], ii) for ii in range(8)], dim=1)
print("translation invariant:", equivariant_linear_transform, sep="\n")

generic:
tensor([[0],
        [1],
        [2],
        [3],
        [4],
        [5],
        [6],
        [7]])
translation invariant:
tensor([[0, 7, 6, 5, 4, 3, 2, 1],
        [1, 0, 7, 6, 5, 4, 3, 2],
        [2, 1, 0, 7, 6, 5, 4, 3],
        [3, 2, 1, 0, 7, 6, 5, 4],
        [4, 3, 2, 1, 0, 7, 6, 5],
        [5, 4, 3, 2, 1, 0, 7, 6],
        [6, 5, 4, 3, 2, 1, 0, 7],
        [7, 6, 5, 4, 3, 2, 1, 0]])


In [27]:
# the equivalent of torch.nn.Linear, but for a 1-dimensional convolution
conv_layer = torch.nn.Conv1d(in_channels=1, out_channels=1, kernel_size=3)

conv_layer.weight  # aka kernel

Parameter containing:
tensor([[[-0.4791, -0.4836, -0.0016]]], requires_grad=True)

In [29]:
conv_kernel_as_vector = torch.hstack([conv_layer.weight[0][0], torch.zeros(5)])
print("convolution vector:", conv_kernel_as_vector, sep="\n")
conv_layer_as_matrix = torch.stack([torch.roll(conv_kernel_as_vector, ii) for ii in range(8)], dim=0)
print("convolution matrix:", conv_layer_as_matrix, sep="\n")

convolution vector:
tensor([-0.4791, -0.4836, -0.0016,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
       grad_fn=<CatBackward0>)
convolution matrix:
tensor([[-0.4791, -0.4836, -0.0016,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000, -0.4791, -0.4836, -0.0016,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000, -0.4791, -0.4836, -0.0016,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000, -0.4791, -0.4836, -0.0016,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000, -0.4791, -0.4836, -0.0016,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000, -0.4791, -0.4836, -0.0016],
        [-0.0016,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000, -0.4791, -0.4836],
        [-0.4836, -0.0016,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000, -0.4791]],
       grad_fn=<StackBackward0>)


In [30]:
# a sample image
display.Image(url="https://distill.pub/2018/building-blocks/examples/input_images/dog_cat.jpeg")

In [31]:
# from https://distill.pub/2018/building-blocks/
display.Image(url="https://fsdl-public-assets.s3.us-west-2.amazonaws.com/distill-feature-attrib.png", width=1024)

In [32]:
layers = ["conv2d0", "conv2d1", "conv2d2", "mixed3a", "mixed3b"]
layer = layers[1]
idx = 52

weight_explorer = display.IFrame(
    src=f"https://storage.googleapis.com/distill-circuits/inceptionv1-weight-explorer/{layer}_{idx}.html", width=1024, height=720)
weight_explorer.iframe = 'style="background: #FFF";\n><'.join(weight_explorer.iframe.split("><"))  # inject background color
weight_explorer

## Applying convolutions to handwritten characeters: CNN's on EMNIST

In [34]:
import text_recognizer.models

text_recognizer.models.CNN??

## EMNIST dataset

In [36]:
import text_recognizer.data

emnist = text_recognizer.data.EMNIST()  # configure
print(emnist.__doc__)

EMNIST dataset of handwritten characters and digits.

    "The EMNIST dataset is a set of handwritten character digits derived from the NIST Special Database 19
    and converted to a 28x28 pixel image format and dataset structure that directly matches the MNIST dataset."
    From https://www.nist.gov/itl/iad/image-group/emnist-dataset

    The data split we will use is
    EMNIST ByClass: 814,255 characters. 62 unbalanced classes.
    


In [43]:
emnist.prepare_data()  # download, save to disk
emnist.setup()  # create torch.utils.data.Datasets, do train/val split

709MB [00:21, 34.6MB/s]                               


Computing SHA-256...
Unzipping EMNIST...
Loading training data from .mat file
Balancing classes to reduce amount of data
Saving to HDF5 in a compressed format...
Saving essential dataset parameters to text_recognizer/data...
Cleaning up...


In [44]:
!echo {emnist.data_dirname()}
!ls {emnist.data_dirname()}
!ls {emnist.data_dirname() / "raw" / "emnist"}

/home/jobquiroz/full_stack_deep_learning/data
downloaded  processed  raw
metadata.toml  readme.md


In [45]:
emnist

EMNIST Dataset
Num classes: 83
Mapping: ['<B>', '<S>', '<E>', '<P>', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', ' ', '!', '"', '#', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', ':', ';', '?']
Dims: (1, 28, 28)
Train/val/test sizes: 260212, 65054, 53988
Batch x stats: (torch.Size([128, 1, 28, 28]), torch.float32, tensor(0.), tensor(0.1693), tensor(0.3258), tensor(1.))
Batch y stats: (torch.Size([128]), torch.int64, tensor(4), tensor(65))

In [46]:
xs, ys = next(iter(emnist.train_dataloader()))

In [47]:
import wandb

idx = random.randint(0, len(xs) - 1)

print(emnist.mapping[ys[idx]])
wandb.Image(xs[idx]).image

Y


Now we have a data config

In [48]:
data_config = emnist.config()

cnn = text_recognizer.models.CNN(data_config)
cnn  # reveals the nn.Modules attached to our nn.Module

CNN(
  (conv1): ConvBlock(
    (conv): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (relu): ReLU()
  )
  (conv2): ConvBlock(
    (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (relu): ReLU()
  )
  (dropout): Dropout(p=0.25, inplace=False)
  (max_pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=12544, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=83, bias=True)
)

In [49]:
idx = random.randint(0, len(xs) - 1)
outs = cnn(xs[idx:idx+1])

print("output:", emnist.mapping[torch.argmax(outs)])
wandb.Image(xs[idx]).image

output: X


In [50]:
torch.flatten(torch.Tensor([[1, 2], [3, 4]]))

tensor([1., 2., 3., 4.])

In [51]:
[p.numel() for p in cnn.parameters()]  # conv weight + bias, conv weight + bias, fc weight + bias, fc weight + bias

[576, 64, 36864, 64, 1605632, 128, 10624, 83]

In [52]:
biggest_layer = [p for p in cnn.parameters() if p.numel() == max(p.numel() for p in cnn.parameters())][0]
biggest_layer.shape, cnn.fc_input_dim

(torch.Size([128, 12544]), 12544)

In [53]:
# for the Linear layers, number of multiplications per input == nparams
cnn.fc1.weight.numel()

1605632

In [54]:
# for the Conv2D layers, it's more complicated

def approx_conv_multiplications(kernel_shape, input_size=(64, 28, 28)):  # this is a rough and dirty approximation
    num_kernel_elements = 1
    for dimension in kernel_shape[-3:]:
        num_kernel_elements *= dimension
    num_input_channels, num_kernels = input_size[0], kernel_shape[0]
    num_spatial_applications = ((input_size[1] - kernel_shape[-2] + 1) * (input_size[2] - kernel_shape[-1] + 1))
    mutliplications_per_kernel = num_spatial_applications * num_kernel_elements * num_input_channels
    return mutliplications_per_kernel * num_kernels

In [55]:
approx_conv_multiplications(cnn.conv2.conv.weight.shape)

1594884096

In [56]:
# ratio of multiplications in the convolution to multiplications in the fully-connected layer is huge!
approx_conv_multiplications(cnn.conv2.conv.weight.shape) // cnn.fc1.weight.numel()

993

In [57]:
%run training/run_experiment.py --help

usage: run_experiment.py [--logger [LOGGER]]
                         [--checkpoint_callback [CHECKPOINT_CALLBACK]]
                         [--enable_checkpointing [ENABLE_CHECKPOINTING]]
                         [--default_root_dir DEFAULT_ROOT_DIR]
                         [--gradient_clip_val GRADIENT_CLIP_VAL]
                         [--gradient_clip_algorithm GRADIENT_CLIP_ALGORITHM]
                         [--process_position PROCESS_POSITION]
                         [--num_nodes NUM_NODES]
                         [--num_processes NUM_PROCESSES] [--devices DEVICES]
                         [--gpus GPUS] [--auto_select_gpus [AUTO_SELECT_GPUS]]
                         [--tpu_cores TPU_CORES] [--ipus IPUS]
                         [--log_gpu_memory LOG_GPU_MEMORY]
                         [--progress_bar_refresh_rate PROGRESS_BAR_REFRESH_RATE]
                         [--enable_progress_bar [ENABLE_PROGRESS_BAR]]
                         [--overfit_batches OVERFIT_BATCHES]
   

In [60]:
!python training/run_experiment.py --help --model_class CNN --data_class EMNIST  | tail -n 25

                        .. deprecated:: v1.5 Trainer argument
                        ``terminate_on_nan`` was deprecated in v1.5 and will
                        be removed in 1.7. Please use ``detect_anomaly``
                        instead.

Data Args:
  --batch_size BATCH_SIZE
                        Number of examples to operate on per forward step.
                        Default is 128.
  --num_workers NUM_WORKERS
                        Number of additional processes to load data. Default
                        is 4.

Model Args:
  --conv_dim CONV_DIM
  --fc_dim FC_DIM
  --fc_dropout FC_DROPOUT

LitModel Args:
  --optimizer OPTIMIZER
                        optimizer class from torch.optim
  --lr LR
  --one_cycle_max_lr ONE_CYCLE_MAX_LR
  --one_cycle_total_steps ONE_CYCLE_TOTAL_STEPS
  --loss LOSS           loss function from torch.nn.functional


In [61]:
gpus = int(torch.cuda.is_available())  # use GPUs if they're available

%run training/run_experiment.py --model_class CNN --data_class EMNIST --gpus {gpus}

Missing logger folder: training/logs/lightning_logs
Trainer already configured with model summary callbacks: [<class 'pytorch_lightning.callbacks.model_summary.ModelSummary'>]. Skipping setting a default `ModelSummary` callback.
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name           | Type      | Params
---------------------------------------------
0 | model          | CNN       | 1.7 M 
1 | model.conv1    | ConvBlock | 640   
2 | model.conv2    | ConvBlock | 36.9 K
3 | model.dropout  | Dropout   | 0     
4 | model.max_pool | MaxPool2d | 0     
5 | model.fc1      | Linear    | 1.6 M 
6 | model.fc2      | Linear    | 10.7 K
7 | train_acc      | Accuracy  | 0     
8 | val_acc        | Accuracy  | 0     
9 | test_acc       | Accuracy  | 0     
---------------------------------------------
1.7 M     Trainable params
0         Non-trainable 

Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: 0it [00:00, ?it/s]

Best model saved at: /home/jobquiroz/full_stack_deep_learning/lab02/training/logs/lightning_logs/version_0/epoch=0000-validation.loss=0.574.ckpt


───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test/acc            0.7847299575805664
        test/loss           0.5782811045646667
───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


In [62]:
# we use a sequence of bash commands to get the latest checkpoint's filename
#  by hand, you can just copy and paste it

list_all_log_files = "find training/logs/lightning_logs"  # find avoids issues with \n in filenames
filter_to_ckpts = "grep \.ckpt$"  # regex match on end of line
sort_version_descending = "sort -Vr"  # uses "version" sorting (-V) and reverses (-r)
take_first = "head -n 1"  # the first n elements, n=1

latest_ckpt, = ! {list_all_log_files} | {filter_to_ckpts} | {sort_version_descending} | {take_first}
latest_ckpt

'training/logs/lightning_logs/version_0/epoch=0000-validation.loss=0.574.ckpt'

In [63]:
!find training/logs/lightning_logs/

training/logs/lightning_logs/
training/logs/lightning_logs/version_0
training/logs/lightning_logs/version_0/hparams.yaml
training/logs/lightning_logs/version_0/epoch=0000-validation.loss=0.574.ckpt
training/logs/lightning_logs/version_0/events.out.tfevents.1661450371.instance-1.29046.0
training/logs/lightning_logs/version_0/events.out.tfevents.1661450411.instance-1.29046.1


In [67]:
import training.util
from argparse import Namespace


# if you change around model/data args in the command above, add them here
#  tip: define the arguments as variables, like we've done for gpus
#       and then add those variables to this dict so you don't need to
#       remember to update/copy+paste

args = Namespace(**{
    "model_class": "CNN",
    "data_class": "EMNIST"})


_, cnn = training.util.setup_data_and_model_from_args(args)

reloaded_model = text_recognizer.lit_models.BaseLitModel.load_from_checkpoint(
   latest_ckpt, args=args, model=cnn)

In [68]:
idx = random.randint(0, len(xs) - 1)
outs = reloaded_model(xs[idx:idx+1])

print("output:", emnist.mapping[torch.argmax(outs)])
wandb.Image(xs[idx]).image

output: E


In [69]:
latest_ckpt, = ! {list_all_log_files} | {filter_to_ckpts} | {sort_version_descending} | {take_first}


# and we can change the training hyperparameters, like batch size
%run training/run_experiment.py --model_class CNN --data_class EMNIST --gpus {gpus} \
  --batch_size 64 --load_checkpoint {latest_ckpt}

Trainer already configured with model summary callbacks: [<class 'pytorch_lightning.callbacks.model_summary.ModelSummary'>]. Skipping setting a default `ModelSummary` callback.
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name           | Type      | Params
---------------------------------------------
0 | model          | CNN       | 1.7 M 
1 | model.conv1    | ConvBlock | 640   
2 | model.conv2    | ConvBlock | 36.9 K
3 | model.dropout  | Dropout   | 0     
4 | model.max_pool | MaxPool2d | 0     
5 | model.fc1      | Linear    | 1.6 M 
6 | model.fc2      | Linear    | 10.7 K
7 | train_acc      | Accuracy  | 0     
8 | val_acc        | Accuracy  | 0     
9 | test_acc       | Accuracy  | 0     
---------------------------------------------
1.7 M     Trainable params
0         Non-trainable params
1.7 M     Total params
6.616     Total estima

Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: 0it [00:00, ?it/s]

Best model saved at: /home/jobquiroz/full_stack_deep_learning/lab02/training/logs/lightning_logs/version_1/epoch=0000-validation.loss=0.536.ckpt


───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test/acc            0.7911202311515808
        test/loss           0.5394253730773926
───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


## EMNISTLines

Let's kick the realism up one notch by building lines of text out of our characters: synthesizing data for our model.

To build fake handwriting, we'll combine two things: real handwritten letters and real text

In [70]:
from text_recognizer.data.sentence_generator import SentenceGenerator

sentence_generator = SentenceGenerator()

SentenceGenerator.__doc__

[nltk_data] Downloading package brown to /home/jobquiroz/full_stack_de
[nltk_data]     ep_learning/data/downloaded/nltk...
[nltk_data]   Unzipping corpora/brown.zip.


'Generate text sentences using the Brown corpus.'

In [99]:
print(*[sentence_generator.generate(max_length=16) for _ in range(5)], sep="\n")

new members
sport There
were clean well
made me
McFeeley and


In [100]:
emnist_lines = text_recognizer.data.EMNISTLines()  # configure
emnist_lines.__doc__

'EMNIST Lines dataset: synthetic handwriting lines dataset made from EMNIST characters.'

In [101]:
emnist_lines.prepare_data()  # download, save to disk
emnist_lines.setup()  # create torch.utils.data.Datasets, do train/val split
emnist_lines

EMNISTLinesDataset generating data for train...
EMNISTLinesDataset generating data for val...
EMNISTLinesDataset generating data for test...
EMNISTLinesDataset loading data from HDF5...


EMNIST Lines Dataset
Min overlap: 0
Max overlap: 0.33
Num classes: 83
Dims: (1, 28, 896)
Output dims: (32, 1)
Train/val/test sizes: 10000, 2000, 2000
Batch x stats: (torch.Size([128, 1, 28, 896]), torch.float32, 0.0, 0.07364349067211151, 0.23208004236221313, 1.0)
Batch y stats: (torch.Size([128, 32]), torch.int64, 3, 66)

In [105]:
line_xs, line_ys = next(iter(emnist_lines.val_dataloader()))
line_xs.shape, line_ys.shape

(torch.Size([128, 1, 28, 896]), torch.Size([128, 32]))

In [106]:
def read_line_labels(labels):
    return [emnist_lines.mapping[label] for label in labels]

idx = random.randint(0, len(line_xs) - 1)

print("-".join(read_line_labels(line_ys[idx])))
wandb.Image(line_xs[idx]).image

d-o-e-s-<P>-<P>-<P>-<P>-<P>-<P>-<P>-<P>-<P>-<P>-<P>-<P>-<P>-<P>-<P>-<P>-<P>-<P>-<P>-<P>-<P>-<P>-<P>-<P>-<P>-<P>-<P>-<P>


## LineCNNSimple

In [107]:
line_cnn = text_recognizer.models.LineCNNSimple(emnist_lines.config())
line_cnn

LineCNNSimple(
  (cnn): CNN(
    (conv1): ConvBlock(
      (conv): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (relu): ReLU()
    )
    (conv2): ConvBlock(
      (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (relu): ReLU()
    )
    (dropout): Dropout(p=0.25, inplace=False)
    (max_pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (fc1): Linear(in_features=12544, out_features=128, bias=True)
    (fc2): Linear(in_features=128, out_features=83, bias=True)
  )
)

In [108]:
idx = random.randint(0, len(line_xs) - 1)

outs, = line_cnn(line_xs[idx:idx+1])
preds = torch.argmax(outs, 0)

print("-".join(read_line_labels(preds)))
wandb.Image(line_xs[idx]).image

+-H-H-H-H-h-H-v-H-H-H-H-H-v-v-H-H-v-H-H-H-H-B-H-v-v-v-H-v-H-H-H


In [109]:
%run training/run_experiment.py --model_class LineCNNSimple --data_class EMNISTLines \
  --batch_size 32 --gpus {gpus} --max_epochs 2

Trainer already configured with model summary callbacks: [<class 'pytorch_lightning.callbacks.model_summary.ModelSummary'>]. Skipping setting a default `ModelSummary` callback.
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


EMNISTLinesDataset loading data from HDF5...


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type          | Params
--------------------------------------------
0 | model     | LineCNNSimple | 1.7 M 
1 | model.cnn | CNN           | 1.7 M 
2 | train_acc | Accuracy      | 0     
3 | val_acc   | Accuracy      | 0     
4 | test_acc  | Accuracy      | 0     
--------------------------------------------
1.7 M     Trainable params
0         Non-trainable params
1.7 M     Total params
6.616     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

...

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


EMNISTLinesDataset loading data from HDF5...


Testing: 0it [00:00, ?it/s]

Best model saved at: /home/jobquiroz/full_stack_deep_learning/lab02/training/logs/lightning_logs/version_2/epoch=0001-validation.loss=1.380.ckpt


───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test/acc            0.6737968921661377
        test/loss           1.4335458278656006
───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


In [110]:
# if you change around model/data args in the command above, add them here
#  tip: define the arguments as variables, like we've done for gpus
#       and then add those variables to this dict so you don't need to
#       remember to update/copy+paste

args = Namespace(**{
    "model_class": "LineCNNSimple",
    "data_class": "EMNISTLines"})


_, line_cnn = training.util.setup_data_and_model_from_args(args)

latest_ckpt, = ! {list_all_log_files} | {filter_to_ckpts} | {sort_version_descending} | {take_first}
print(latest_ckpt)

reloaded_lines_model = text_recognizer.lit_models.BaseLitModel.load_from_checkpoint(
   latest_ckpt, args=args, model=line_cnn)

training/logs/lightning_logs/version_2/epoch=0001-validation.loss=1.380.ckpt


In [112]:
idx = random.randint(0, len(line_xs) - 1)

outs, = reloaded_lines_model(line_xs[idx:idx+1])
preds = torch.argmax(outs, 0)

print("-".join(read_line_labels(preds)))
wandb.Image(line_xs[idx]).image

a- -g-e-n-e-n-l-<P>-<P>-<P>-<P>-<P>-<P>-<P>-<P>-<P>-<P>-<P>-<P>-<P>-<P>-<P>-<P>-<P>-<P>-<P>-<P>-<P>-<P>-<P>-<P>


In [113]:
padding_token = emnist_lines.emnist.inverse_mapping["<P>"]
torch.sum(line_ys == padding_token) / line_ys.numel()

tensor(0.5359)